<a href="https://colab.research.google.com/github/ch23s020/assignment2/blob/main/Assignment2_CH23S020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from zipfile import ZipFile

# Mount Google Drive (authentication required only on first run)
drive.mount('/content/drive')

# Specify paths
zip_path = "/content/drive/MyDrive/nature_12K.zip"
target_dir = "/content/drive/MyDrive/Assignment_2"  # Note the double underscore

try:
  # Create the target directory (if it doesn't exist)
  !mkdir -p {target_dir}

  # Extract the zip file
  with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

  print("Unzipped successfully!")

except Exception as e:
  print(f"Error unzipping: {e}")

# Unmount Google Drive (optional)
#drive.unmount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import numpy as np

In [ ]:
class CNN_i_Naturalist(nn.Module):
    def __init__(self, num_classes, num_filters, filter_size, activation, filter_org, use_batchnorm, use_dropout):
        super(CNN_i_Naturalist, self).__init__()
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.activation = activation
        self.filter_org = filter_org
        self.use_batchnorm = use_batchnorm
        self.use_dropout = use_dropout

        # Convolution layers
        layers = []
        in_channels = 3 # Assuming RGB images
        prev_output_size = 224  # Initial image size, adjust as necessary
        for i in range(len(num_filters)):
            layers.append(nn.Conv2d(in_channels, num_filters[i], filter_size[i]))
            if use_batchnorm:
                layers.append(nn.BatchNorm2d(num_filters[i]))
            if activation == 'ReLU':
                layers.append(nn.ReLU())
            elif activation == 'GELU':
                layers.append(nn.GELU())
            elif activation == 'SiLU':
                layers.append(nn.SiLU())
            elif activation == 'Mish':
                layers.append(nn.Mish())
            layers.append(nn.MaxPool2d(2, 2))
            if use_dropout:
                layers.append(nn.Dropout(0.2))
            in_channels = num_filters[i]
            # Calculate the output size of this layer
            prev_output_size = (prev_output_size - filter_size[i][0] + 1) // 2
        self.conv_layers = nn.Sequential(*layers)

        # Calculate the input size to the fully connected layers
        self.fc_input_size = in_channels * prev_output_size * prev_output_size

        # Dense layers
        self.fc = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.fc2(x)
        return x

# Hyperparameters
num_classes = 10
num_filters = [32, 64, 128, 256, 512]
filter_size = [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
activation = 'ReLU'  #activation function
filter_org = 'double'  # 'same', 'double', 'halve'
use_batchnorm = True  # Set to True or False
use_dropout = True  # Set to True or False
num_epochs = 2

In [ ]:
layers = []
in_channels = 3
num_classes = 10
num_filters = [32, 64, 128, 256, 512]
 for i in range(len(num_filters)):
            layers.append(nn.Conv2d(in_channels, num_filters[i], filter_size[i]))
            if use_batchnorm:
                layers.append(nn.BatchNorm2d(num_filters[i]))
            if activation == 'ReLU':
                layers.append(nn.ReLU())
            elif activation == 'GELU':
                layers.append(nn.GELU())
            elif activation == 'SiLU':
                layers.append(nn.SiLU())
            elif activation == 'Mish':
                layers.append(nn.Mish())
            layers.append(nn.MaxPool2d(2, 2))
            if use_dropout:
                layers.append(nn.Dropout(0.2))
            in_channels = num_filters[i]